In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# ----------------------------
# 1️⃣ Load & Clean Dataset
# ----------------------------
df = pd.read_csv("survey_data_cleaned.csv")
drop_cols = ['Timestamp', 'Date_Of_Birth', 'Age']
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# ----------------------------
# 2️⃣ Split Target and Features
# ----------------------------
X = df.drop(columns=['Career_Interest'])
y = df['Career_Interest']

print("[INFO] Original Class Distribution:")
print(y.value_counts(), "\n")

# ----------------------------
# 3️⃣ Label Encoding
# ----------------------------
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col].astype(str))
y_encoded = le.fit_transform(y)

# ----------------------------
# 4️⃣ ANOVA Feature Selection
# ----------------------------
F_values, p_values = f_classif(X, y_encoded)
anova_df = pd.DataFrame({'Feature': X.columns, 'F_value': F_values, 'p_value': p_values})
selected_features = anova_df[anova_df['p_value'] < 0.05]['Feature'].tolist()
X_selected = X[selected_features]

print("[INFO] Selected Features via ANOVA (<0.05):")
print(selected_features, "\n")

# ----------------------------
# 5️⃣ Train-Test Split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

print("[INFO] Training Class Distribution:")
print(pd.Series(y_train).value_counts(), "\n")
print("[INFO] Testing Class Distribution:")
print(pd.Series(y_test).value_counts(), "\n")

# ----------------------------
# 6️⃣ Scaling
# ----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------------
# 7️⃣ SMOTE
# ----------------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

print("[INFO] After SMOTE Class Distribution:")
print(pd.Series(y_train_res).value_counts(), "\n")

# ----------------------------
# 8️⃣ Train Models with internal bootstrapping
# ----------------------------
rf = RandomForestClassifier(n_estimators=100, random_state=42, bootstrap=True)
rf.fit(X_train_res, y_train_res)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_res, y_train_res)

ensemble = VotingClassifier(estimators=[('RF', rf), ('XGB', xgb)], voting='soft')
ensemble.fit(X_train_res, y_train_res)

# ----------------------------
# 9️⃣ Prediction
# ----------------------------
y_pred_rf = rf.predict(X_test_scaled)
y_pred_xgb = xgb.predict(X_test_scaled)
y_pred_ens = ensemble.predict(X_test_scaled)

# ----------------------------
# 🔟 Evaluation
# ----------------------------
def print_report(y_true, y_pred, model_name):
    print(f"============ {model_name} REPORT ============")
    print(classification_report(y_true, y_pred))
    print("Accuracy:", accuracy_score(y_true, y_pred), "\n")

print_report(y_test, y_pred_rf, "RANDOM FOREST")
print_report(y_test, y_pred_xgb, "XGBOOST")
print_report(y_test, y_pred_ens, "ENSEMBLE")


[INFO] Original Class Distribution:
Career_Interest
1    64
0    53
3    51
2    41
Name: count, dtype: int64 

[INFO] Selected Features via ANOVA (<0.05):
['Gender', 'Math', 'Bio', 'Chemistry', 'Physics', 'Business', 'Average_Score', 'Study_Method', 'English_Proficiency', 'Favorite_Subject', 'Building miniatures / models', 'Hackathons / App development projects', 'Programming / coding clubs', 'Volunteering at hospitals, clinics, or NGOs', 'Model building recognition', 'Programming / Coding award'] 

[INFO] Training Class Distribution:
1    51
0    42
3    41
2    33
Name: count, dtype: int64 

[INFO] Testing Class Distribution:
1    13
0    11
3    10
2     8
Name: count, dtype: int64 

[INFO] After SMOTE Class Distribution:
2    51
1    51
0    51
3    51
Name: count, dtype: int64 



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:23:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:23:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


============ RANDOM FOREST REPORT ============
              precision    recall  f1-score   support

           0       0.70      0.64      0.67        11
           1       0.67      0.62      0.64        13
           2       0.46      0.75      0.57         8
           3       0.71      0.50      0.59        10

    accuracy                           0.62        42
   macro avg       0.64      0.63      0.62        42
weighted avg       0.65      0.62      0.62        42

Accuracy: 0.6190476190476191 

============ XGBOOST REPORT ============
              precision    recall  f1-score   support

           0       0.62      0.73      0.67        11
           1       0.78      0.54      0.64        13
           2       0.31      0.50      0.38         8
           3       0.71      0.50      0.59        10

    accuracy                           0.57        42
   macro avg       0.60      0.57      0.57        42
weighted avg       0.63      0.57      0.58        42

Accuracy: 0

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# ----------------------------
# 1️⃣ Load & Clean Dataset
# ----------------------------
df = pd.read_csv("survey_data_cleaned.csv")
drop_cols = ['Timestamp', 'Date_Of_Birth', 'Age']
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# ----------------------------
# 2️⃣ Split Target and Features
# ----------------------------
X = df.drop(columns=['Career_Interest'])
y = df['Career_Interest']

print("[INFO] Original Class Distribution:")
print(y.value_counts(), "\n")

# ----------------------------
# 3️⃣ Label Encoding
# ----------------------------
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col].astype(str))
y_encoded = le.fit_transform(y)

# ----------------------------
# 4️⃣ ANOVA Feature Selection
# ----------------------------
F_values, p_values = f_classif(X, y_encoded)
anova_df = pd.DataFrame({'Feature': X.columns, 'F_value': F_values, 'p_value': p_values})
selected_features = anova_df[anova_df['p_value'] < 0.05]['Feature'].tolist()
X_selected = X[selected_features]

print("[INFO] Selected Features via ANOVA (<0.05):")
print(selected_features, "\n")

# ----------------------------
# 5️⃣ Train-Test Split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

print("[INFO] Training Class Distribution:")
print(pd.Series(y_train).value_counts(), "\n")
print("[INFO] Testing Class Distribution:")
print(pd.Series(y_test).value_counts(), "\n")

# ----------------------------
# 6️⃣ Scaling
# ----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------------
# 7️⃣ Feature-Weighted SMOTE
# ----------------------------
importance_dict = dict(zip(selected_features, F_values / F_values.sum()))
feature_weights = np.array([importance_dict[f] for f in selected_features])

X_weighted = X_train_scaled * feature_weights
smote_fw = SMOTE(random_state=42)
X_res, y_res = smote_fw.fit_resample(X_weighted, y_train)
X_train_res_scaled = X_res / feature_weights

print("[INFO] After FW-SMOTE Class Distribution:")
print(pd.Series(y_res).value_counts(), "\n")

# ----------------------------
# 8️⃣ Train Models with internal bootstrapping
# ----------------------------
rf = RandomForestClassifier(n_estimators=100, random_state=42, bootstrap=True)
rf.fit(X_train_res_scaled, y_res)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_res_scaled, y_res)

ensemble = VotingClassifier(estimators=[('RF', rf), ('XGB', xgb)], voting='soft')
ensemble.fit(X_train_res_scaled, y_res)

# ----------------------------
# 9️⃣ Prediction
# ----------------------------
y_pred_rf = rf.predict(X_test_scaled)
y_pred_xgb = xgb.predict(X_test_scaled)
y_pred_ens = ensemble.predict(X_test_scaled)

# ----------------------------
# 🔟 Evaluation
# ----------------------------
def print_report(y_true, y_pred, model_name):
    print(f"============ {model_name} REPORT ============")
    print(classification_report(y_true, y_pred))
    print("Accuracy:", accuracy_score(y_true, y_pred), "\n")

print_report(y_test, y_pred_rf, "RANDOM FOREST")
print_report(y_test, y_pred_xgb, "XGBOOST")
print_report(y_test, y_pred_ens, "ENSEMBLE")


[INFO] Original Class Distribution:
Career_Interest
1    64
0    53
3    51
2    41
Name: count, dtype: int64 

[INFO] Selected Features via ANOVA (<0.05):
['Gender', 'Math', 'Bio', 'Chemistry', 'Physics', 'Business', 'Average_Score', 'Study_Method', 'English_Proficiency', 'Favorite_Subject', 'Building miniatures / models', 'Hackathons / App development projects', 'Programming / coding clubs', 'Volunteering at hospitals, clinics, or NGOs', 'Model building recognition', 'Programming / Coding award'] 

[INFO] Training Class Distribution:
1    51
0    42
3    41
2    33
Name: count, dtype: int64 

[INFO] Testing Class Distribution:
1    13
0    11
3    10
2     8
Name: count, dtype: int64 

[INFO] After FW-SMOTE Class Distribution:
2    51
1    51
0    51
3    51
Name: count, dtype: int64 



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:23:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:23:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


============ RANDOM FOREST REPORT ============
              precision    recall  f1-score   support

           0       0.67      0.55      0.60        11
           1       0.60      0.69      0.64        13
           2       0.55      0.75      0.63         8
           3       0.71      0.50      0.59        10

    accuracy                           0.62        42
   macro avg       0.63      0.62      0.62        42
weighted avg       0.63      0.62      0.62        42

Accuracy: 0.6190476190476191 

============ XGBOOST REPORT ============
              precision    recall  f1-score   support

           0       0.62      0.73      0.67        11
           1       0.70      0.54      0.61        13
           2       0.31      0.50      0.38         8
           3       0.67      0.40      0.50        10

    accuracy                           0.55        42
   macro avg       0.57      0.54      0.54        42
weighted avg       0.60      0.55      0.55        42

Accuracy: 0